In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LinearRegression

In [35]:
df=pd.read_csv("WIKI-PRICES.csv")

In [36]:
df.head()

,ticker,date,open,high,low,close,volume,ex-dividend,split_ratio,adj_open,adj_high,adj_low,adj_close,adj_volume
0,ZUMZ,2018-03-27,24.65,24.65,23.3500,23.60,403884.0,0.0,1.0,24.65,24.65,23.3500,23.60,403884.0
1,ZUMZ,2018-03-26,23.75,24.80,23.7000,24.65,375320.0,0.0,1.0,23.75,24.80,23.7000,24.65,375320.0
2,ZUMZ,2018-03-23,23.55,24.20,23.4500,23.55,301584.0,0.0,1.0,23.55,24.20,23.4500,23.55,301584.0
3,ZUMZ,2018-03-22,23.90,24.35,23.3000,23.35,269607.0,0.0,1.0,23.90,24.35,23.3000,23.35,269607.0
4,ZUMZ,2018-03-21,23.80,24.60,23.6058,23.95,354092.0,0.0,1.0,23.80,24.60,23.6058,23.95,354092.0


In [37]:
# High-low percentage
df['HL_PCT'] = (df['adj_high'] - df['adj_close']) / df['adj_close'] * 100

# Percentage change ((new - old) / old * 100)
df['PCT_change'] = (df['adj_close'] - df['adj_open']) / df['adj_open'] * 100

# Define useful columns
columns = [
    "date",
    'adj_close',
    'adj_volume',
    'HL_PCT',
    'PCT_change',
]

df = df[columns]

df.head()

,date,adj_close,adj_volume,HL_PCT,PCT_change
0,2018-03-27,23.60,403884.0,4.449153,-4.259635
1,2018-03-26,24.65,375320.0,0.608519,3.789474
2,2018-03-23,23.55,301584.0,2.760085,0.000000
3,2018-03-22,23.35,269607.0,4.282655,-2.301255
4,2018-03-21,23.95,354092.0,2.713987,0.630252


In [38]:
df = df.set_index('date')

In [39]:
df.head()

,adj_close,adj_volume,HL_PCT,PCT_change
date,,,,
2018-03-27,23.60,403884.0,4.449153,-4.259635
2018-03-26,24.65,375320.0,0.608519,3.789474
2018-03-23,23.55,301584.0,2.760085,0.000000
2018-03-22,23.35,269607.0,4.282655,-2.301255
2018-03-21,23.95,354092.0,2.713987,0.630252


In [42]:
df.index.name = None
df.head()

,adj_close,adj_volume,HL_PCT,PCT_change
2018-03-27,23.60,403884.0,4.449153,-4.259635
2018-03-26,24.65,375320.0,0.608519,3.789474
2018-03-23,23.55,301584.0,2.760085,0.000000
2018-03-22,23.35,269607.0,4.282655,-2.301255
2018-03-21,23.95,354092.0,2.713987,0.630252


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 2018-03-27 to 1994-01-04
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   adj_close   10000 non-null  float64
 1   adj_volume  10000 non-null  float64
 2   HL_PCT      10000 non-null  float64
 3   PCT_change  10000 non-null  float64
dtypes: float64(4)
memory usage: 390.6+ KB


In [44]:
len(df)

10000

In [45]:
forecast_col="adj_close"
df.fillna(-9999, inplace=True)
import math
# amount of days in advance
forecast_out = int(math.ceil(0.01*len(df)))
forecast_out

100

In [46]:
# set the label to the shift of the forecast col
df['label'] = df[forecast_col].shift(-forecast_out)
df.dropna(inplace=True)
df.head()

,adj_close,adj_volume,HL_PCT,PCT_change,label
2018-03-27,23.60,403884.0,4.449153,-4.259635,17.40
2018-03-26,24.65,375320.0,0.608519,3.789474,17.95
2018-03-23,23.55,301584.0,2.760085,0.000000,18.50
2018-03-22,23.35,269607.0,4.282655,-2.301255,18.05
2018-03-21,23.95,354092.0,2.713987,0.630252,17.90


In [47]:
# X feature
X = np.array(df.drop(['label'],axis=1))

# y labels
y = np.array(df['label'])

# Scale up X
X = preprocessing.scale(X)
y = np.array(df['label'])

# Create the train and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [48]:
clf = LinearRegression()
# Train the model
clf.fit(X_train, y_train)
# Test the accuracy
clf.score(X_test, y_test)

0.7357015764902737

In [49]:
from sklearn import svm
clf = svm.SVR()
# Train the model
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.7381208061031114

In [50]:
forecast_set = clf.predict(X_lately)
forecast_set

array([29.31261565, 27.83618275, 33.82621029, 32.73827227, 29.34688757,
       28.06010808, 30.62841246, 27.96577775, 28.61937128, 24.7624322 ,
       36.79443113, 30.20839629, 28.33014542, 30.01186307, 30.37858314,
       32.48052923, 28.9962659 , 35.44171704, 30.59692273, 32.22754153,
       28.10796039, 32.15022847, 31.34393769, 30.22241079, 28.54121652,
       31.37611532, 27.00831158, 29.26523633, 26.3636929 , 26.56799114,
       29.84044643, 27.35771031, 27.8886312 , 27.82433249, 33.78187886,
       28.72445917, 29.07648966, 35.62786643, 29.38883643, 27.24700644,
       29.48776636, 29.84740424, 26.791232  , 27.58349445, 26.92701925,
       28.13634784, 26.29087024, 35.29670348, 30.48977076, 30.66628233,
       31.77528914, 34.42550089, 30.26167602, 30.06475988, 28.80903819,
       32.18022468, 27.22156877, 28.74891522, 29.79836812, 29.36441504,
       26.71142948, 28.20513673, 27.36227451, 27.7530037 , 27.02292671,
       27.50828521, 29.06957247, 27.65590176, 28.87717314, 30.67

In [56]:
import datetime
import time
df['Forecast'] = np.nan
last_date = df.iloc[-1].name
last_unix  = last_date.timestamp()
one_day = 86400
next_unix = last_unix+one_day

for i in forecast_set:
    next_date = datetime.datetime.fromtimestamp(next_unix)
    next_unix += one_day
    df.loc[next_date] = [np.nan for _ in range(len(df.columns)-1)]+[i]

print(df.head())


AttributeError: 'str' object has no attribute 'timestamp'